%md
## ベクター検索エンドポイントとベクター検索インデックスを作成

手順の参考：[Azure｜ベクター検索インデックスを作成してクエリを実行する方法](https://learn.microsoft.com/ja-jp/azure/databricks/generative-ai/create-query-vector-search)

ベクター検索を使うためには、主に次の3つの要素が必要です：

1. ベクター検索エンドポイント（検索を提供するAPI窓口）
2. ベクター検索インデックス（検索対象のデータ本体）
3. 埋め込み（Embedding）モデル（テキストを数値ベクトルに変換）

### ベクター検索エンドポイント

参考：[Microsoft｜モザイク AI ベクター検索を設定する方法](https://learn.microsoft.com/ja-jp/azure/databricks/generative-ai/vector-search#how-to-set-up-mosaic-ai-vector-search)

いわば「検索サーバー」や「検索APIの入り口」。

インデックスをホストしている実体です。

REST API や SDK を通じて、検索クエリを送ったり、インデックスを更新したりできます。

例：「この文章に似たドキュメントを探して」と問い合わせると、類似データを返す役割。

エンドポイントにはは2種類ある。

| 種類                  | 特徴                       |
| ------------------- | ------------------------ |
| **ストレージ最適化エンドポイント** | インデックス削除時に自動スケールダウン（節約型） |
| **標準エンドポイント**       | 自動スケールダウンなし       |


### ベクター検索インデックスを作成

検索対象となる 「埋め込みベクトルを格納したデータ構造」 です。
SQLでいうテーブルに似ていますが、「類似検索（Approximate Nearest Neighbor, ANN）」に最適化されています。

- ソースは Deltaテーブル（＝元データ）
- DatabricksがそのDeltaをもとに、検索用のインデックスを構築します
- Unity Catalogで管理され、権限やデータガバナンスも統合されています

**目的**

「クエリ（質問）に似たベクトルをすばやく探す」こと。
つまり、自然言語やドキュメントの**意味的な検索（semantic search）**が可能になります。

#### 作成してみる

ベクトル化されたPDFのテキストデータを格納したテーブルからベクトル検索インデックスを作成

![chunk_embeddings-Databricks.png](./ベクトル検索インデックスを作成.png "chunk_embeddings_-_Databricks.png")


設定値

| 項目                                           | 選択した値                             | 備考・解説                                 |
| -------------------------------------------- | ------------------------------- | ------------------------------------- |
| **名前 (Name)**                                | `mydocs_index` |     |
| **主キー (Primary key)**                        | `id`                            | Deltaテーブル作成時に付与（自動採番）。差分更新に必要。        |
| **同期する列 (Columns to sync)**                  | 空欄（すべて同期）                       | ベクターインデックスと自動同期する列を指定する。小規模データなら全列同期でOK。特にFree環境では管理が楽。       |
| **埋め込みソース (Embedding source)**               | **「コンピューティング埋め込み」**          | 自前でベクトル化コードを書く必要なし。GUIだけで完結。          |
| **埋め込みモデル (Embedding model)**                | `databricks-bge-large-en`       | Free Editionでも利用可能。日本語もある程度対応。        |
| **埋め込み対象列 (Text column)**                    | `content`（PDFテキスト格納列）           | PDFから抽出して保存したテキストの列を選ぶ。                  |
| **計算された埋め込みの同期 (Store computed embeddings)** |  **ON（有効）**                    | 埋め込み結果をUnity Catalogに保存しておくと後から再利用可能。 |
| **ベクトル検索エンドポイント (Vector Search Endpoint)**   | 既存のもの（例：`vs-endpoint-01`）       | 無料環境では1つ作成して使い回すのがシンプル。               |
| **同期モード (Sync mode)**                        |  **トリガー実行**                   | 手動で同期するほうがFree環境ではコスト＆安定性面で安全。        |
| **エンドポイント種別**                                |  **ストレージ最適化エンドポイント**          | 小〜中規模データ向け。クラスタ起動コストなし。Free環境でも動作。    |


**ベクター検索インデッククスには２種類ある**
1. 差分同期インデックス
    - ソーステーブルが変更されると、ベクター検索インデックスに自動的に反映されます。
2. Direct Vector Access Index（直接アクセス型インデックス）
    - ユーザーが明示的に、REST API または Python SDK を使用してベクターインデックスを更新する必要があります。

**エラーTips**

以下のエラーが発生したが、私の場合はサーバレスSQLウェアハウスを起動すれば解決した。

> このテーブルからインデックスを作成するには、チェンジデータフィードを有効にする必要があります。チェンジデータフィードの詳細については、デルタドキュメント をご覧ください。 チェンジデータフィードを更新するにはアクティブなコンピュートが必要です。DBRバージョン11.3以降もしくはPro/サーバーレスウェアハウスでクラスターを開始してください


## AI Playgroundで作成したインデックスを試す

この機能により、コストを抑えつつ、許容できる回答精度を出すのはどのLLMモデルか見ることができる。


今回はFreeEditionでコストの考慮は不要だった。  
回答は少し遅いが丁寧であるため、今回は左の`GPT OSS 20B`を採用した。

![Playground_-_Databricks.png](./Playground_-_Databricks.png "Playground_-_Databricks.png")